In [1]:
################### 买入后 上涨 大于30 的训练  
import numpy as np
import matplotlib.pyplot  as plt
import pandas as pd
import random

In [19]:
def sigmond(x):
    return 1/(1+np.exp(-x))

def reLu(x):
    return (np.abs(x)+x) / 2.0

def d_reLu(x):
    x[x<=0] = 0
    x[x>1] = 1
    return x

def softmax(x):
    x_row_max = x.max(axis=1).reshape(x.shape[0],1)
    x = x - x_row_max
    x_exp = np.exp(x)
    x_row_exp_sum = x_exp.sum(axis=1).reshape(x.shape[0],1)
    x_softmax = x_exp / x_row_exp_sum
    return x_softmax

def softmax_vector(x):
    x_exp = np.exp(x-x.max())
    return x_exp / x_exp.sum()

######### m 为样本个数 ###############
def J_sigmond(h,y,m):
    J = -y*np.log(h) - (1-y)*np.log(1-h)
    return J.sum() / m

In [29]:
###### 读取数据 ########
stock_data_org = pd.read_csv('stock_data_normal_u_30.csv',header=None)
stock_lable_org = pd.read_csv('stock_data_lable_normal_30.txt',header=None)

In [30]:
print(stock_data_org.shape[0])
print(stock_data_org.shape[0] / 6)
print(stock_lable_org.shape[0])

128094
21349.0
21349


In [33]:
################# 抽取出成功的标签，使成功与失败的比例是1:1.2 #####################
######### 所有的 股票数据，转换成（m,66 的形势）
all_x_data = stock_data_org.values
all_x_data = all_x_data.reshape(-1,66)
all_x_data = np.insert(all_x_data,0,1,axis=1)

################ 为 lable 加上 index
stock_lable_org['index'] = pd.DataFrame(np.arange(stock_lable_org.shape[0] -1))

############### 取出 所有 多空中 为 1 的数据 #####################
stock_lable_1 = stock_lable_org[stock_lable_org[2]==1]
############### 取出 所有 多空中 为 0 的数据 #####################
stock_lable_0 = stock_lable_org[stock_lable_org[2]==0] ##########
############### 对 多空为0 的数据进行抽样，数量为 多空为1 的1.2倍
stock_lable_0_sample = stock_lable_0.sample(n=int(stock_lable_1.shape[0]*1.2),replace=False)
############### 组合 0 的抽样数据与 1 的lable【所有数据】，形成 神经网络 的【训练集】与【测试集】
lable_nn_data = pd.concat([stock_lable_1,stock_lable_0_sample])
lable_nn_data = lable_nn_data.sort_values(by = 'index',ascending=True)
cut_idx = int(lable_nn_data.shape[0] * 0.7)
lable_nn_train = lable_nn_data.iloc[:cut_idx]
lable_nn_test = lable_nn_data.iloc[cut_idx:]
#print(lable_nn_test)
############### 根据 index 取出 所有对应的 股票数据 #####################
train_data_index = lable_nn_train['index'].values
#print(train_data_index)
test_data_index = lable_nn_test['index'].values
#print(test_data_index)
train_data_index = train_data_index.astype(int)
test_data_index = test_data_index.astype(int)
#print(test_data_index)
#print(test_data_index.tolist())
train_data = all_x_data[train_data_index.tolist()]
test_data = all_x_data[test_data_index.tolist()]
#train_data = np.insert(train_data,0,1,axis=1)
#test_data = np.insert(test_data,0,1,axis=1)
y_train = lable_nn_train[2].values.reshape(-1,1)
y_test = lable_nn_test[2].values.reshape(-1,1)

In [13]:
print(train_data.shape[0])
print(y_train.shape[0])
print(test_data.shape[0])
print(y_test.shape[0])

2107
2107
904
904


In [14]:
############## 验证数据是否取正确 ################
for i in range(10):
    random_index = random.randint(0,train_data_index.shape[0]-1)
    check_index = train_data_index[random_index]
    print('输入x',train_data[random_index])
    print('输入y',y_train[random_index])
    print('原有x',all_x_data[check_index])
    print('原有y',stock_lable_org.iloc[check_index])

输入x [ 1.      -1.11226  1.25267  0.18884 -1.30952  1.73918  1.35651 -1.58171
 -1.31456 -1.08132 -1.31979 -1.35028  1.48301 -0.19272  0.99818  1.09831
 -1.00634  0.89323 -0.53975 -0.7068  -0.64295 -0.79291 -0.78043 -0.27806
 -1.49356 -0.62049 -0.67588 -0.94628 -0.1423  -0.2863  -0.36916 -0.53336
 -0.29702 -0.26014 -1.01957 -0.48179 -1.10609 -0.54915  0.16761 -1.01865
  0.44588  0.15757 -0.05114  0.26086  0.26014  0.37075 -0.04818 -0.78236
  0.33794 -0.11483 -1.1408   1.0091   0.87337  0.67217  0.78774  0.78043
  0.55613  0.96359  1.32191  1.09831  0.16066  0.052    0.95278  1.35958
  1.6366   1.36112  1.35028]
输入y [0]
原有x [-1.11226  1.25267  0.18884 -1.30952  1.73918  1.35651 -1.58171 -1.31456
 -1.08132 -1.31979 -1.35028  1.48301 -0.19272  0.99818  1.09831 -1.00634
  0.89323 -0.53975 -0.7068  -0.64295 -0.79291 -0.78043 -0.27806 -1.49356
 -0.62049 -0.67588 -0.94628 -0.1423  -0.2863  -0.36916 -0.53336 -0.29702
 -0.26014 -1.01957 -0.48179 -1.10609 -0.54915  0.16761 -1.01865  0.44588
  0.15

In [15]:
#################读取w1 与w2##################################
w1 = np.loadtxt("init_w1_30.txt",delimiter=',')
w2 = np.loadtxt("init_w2_30.txt",delimiter=',')
w2 = w2.reshape(-1,1)
#################根据文件内容，初始化w1与w2#####################

######### 设置学习率 
n = 0.5
#print(w2[0:10,:])
print(w1[0:10,0:5])
print(w2[0:10])

[[-3.52046151e-01 -4.20313034e-01 -3.21658715e-01  1.88763715e-01
   2.35213841e-01]
 [ 7.38760535e-01  3.21380355e-01  6.15000297e-01  7.29108377e-01
   5.83522937e-01]
 [ 6.21207262e-01  1.51662811e-01 -4.04286707e-01 -5.57458653e-03
   4.07640171e-01]
 [-2.57574159e-01  6.92141231e-01 -3.82890199e-01 -3.97828491e-01
  -5.11410140e-01]
 [ 6.39144123e-01 -3.41104771e-04  6.64647818e-01 -5.91305309e-01
   2.29880085e-02]
 [ 6.78462316e-01 -2.46893800e-01  6.20012334e-01 -3.88554297e-01
  -6.69667824e-01]
 [ 2.30521177e-01 -2.06923360e-01  9.89263758e-01  5.60633349e-01
  -5.26218070e-01]
 [-1.45942469e-01  4.48890837e-01  8.72429921e-02  1.94697984e-01
   9.45396006e-01]
 [ 7.90409931e-01  7.25474716e-01 -9.81728402e-01  6.84653558e-01
  -5.32821582e-01]
 [-6.26349743e-01 -7.59853767e-01 -5.96242506e-01 -5.07400210e-01
   6.34197382e-01]]
[[-1.51762705]
 [-0.51845944]
 [ 0.04497157]
 [ 0.57459626]
 [ 0.87583703]
 [ 1.20368333]
 [ 0.856107  ]
 [ 0.65805428]
 [ 0.19149257]
 [ 1.07027174]

In [21]:
for i in range(2000):

    ############ 隐藏层输入 ############
    hid_input = np.matmul(train_data,w1)  ##### 10000* 200
    ############ 隐藏层输出 ############
    hid_output = np.tanh(hid_input)
    ######## 为隐藏层增加偏执项 ############
    hid_output = np.insert(hid_output,0,1,axis=1)   # 10000* 201
    ############ 最后一层的输入 ############
    hpfs_input = np.matmul(hid_output,w2)
    ############ 最后一层的输出，即正向传播的输出值 ############
    #print(hpfs_input[0:10])
    hpfs_output = sigmond(hpfs_input)
    #print(hpfs_output[0:10])
    ############ 完成正向传播 ############

    ####################### 下面是 用reLu 和 sigmond 的情况 ###########################
    ############ w2的权重误差计算 #######################

    d_cost_hyfs_input = hpfs_output - y_train
    d_cost_w2 = np.matmul(hid_output.T,d_cost_hyfs_input) / 2000
    #print(d_cost_w2.shape)
    #print(d_cost_w2[100:220,:])
    ######## d_cost_w2.shape = 393*10 ############
    #print(d_cost_hyfs_input.shape,w2.shape)
    d_cost_hid_out = np.matmul(d_cost_hyfs_input,w2.T) 
    d_cost_hid_input = d_cost_hid_out*(1-hid_output*hid_output)
    d_cost_hid_input = np.delete(d_cost_hid_input,0,axis=1)
    d_cost_w1 = np.matmul(train_data.T,d_cost_hid_input) / 2000
    print('----------cost is -------------',J_sigmond(hpfs_output,y_train,train_data.shape[0]))
    w1 = w1 - n*d_cost_w1
    w2 = w2 - n*d_cost_w2
    #print('w1 最大值位置',np.where(w1==w1.max()),',w1 最大值',np.max(w1))
    #print('d_w1 最大值位置',np.where(d_cost_w1==d_cost_w1.max()),',d_w1 最大值',np.max(d_cost_w1))
    np.savetxt("init_w1_30.txt",w1,delimiter=',')
    np.savetxt("init_w2_30.txt",w2,delimiter=',')

print('----------------done--------------------------')

----------cost is ------------- 0.8948167639657432
----------cost is ------------- 1.2471225471744636
----------cost is ------------- 4.598193485631797
----------cost is ------------- 4.239395683351772
----------cost is ------------- nan
----------cost is ------------- 1.8566529231111455


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:26: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in multiply


----------cost is ------------- 2.3026402166343236
----------cost is ------------- 2.2471209117591964
----------cost is ------------- 3.814109889138419
----------cost is ------------- 2.000379183372102
----------cost is ------------- 2.81379755666332
----------cost is ------------- 1.9284469271331508
----------cost is ------------- 2.720710614602159
----------cost is ------------- 1.8171689254744947
----------cost is ------------- 2.4508252486872384
----------cost is ------------- 1.7172685672761507
----------cost is ------------- 2.27839221684783
----------cost is ------------- 1.63570398271847
----------cost is ------------- 2.1455644949808534
----------cost is ------------- 1.5620885132164306
----------cost is ------------- 2.0265349570047535
----------cost is ------------- 1.494036374759096
----------cost is ------------- 1.9220352798402827
----------cost is ------------- 1.433770890773288
----------cost is ------------- 1.83404416712755
----------cost is ------------- 1.3808464354

----------cost is ------------- 0.5728928574237456
----------cost is ------------- 0.47875503301655564
----------cost is ------------- 0.5681955411812237
----------cost is ------------- 0.4751329708237512
----------cost is ------------- 0.563575765756581
----------cost is ------------- 0.4715662385660524
----------cost is ------------- 0.5590304304705377
----------cost is ------------- 0.468052625075968
----------cost is ------------- 0.5545565401432102
----------cost is ------------- 0.4645899992646796
----------cost is ------------- 0.5501512049194666
----------cost is ------------- 0.461176309080952
----------cost is ------------- 0.5458116389946345
----------cost is ------------- 0.4578095799079125
----------cost is ------------- 0.5415351586481093
----------cost is ------------- 0.45448791261672
----------cost is ------------- 0.5373191799443534
----------cost is ------------- 0.45120948147314677
----------cost is ------------- 0.5331612164032994
----------cost is ------------- 0.

----------cost is ------------- 0.31476784960326526
----------cost is ------------- 0.27271711678601124
----------cost is ------------- 0.3124215914931717
----------cost is ------------- 0.2707757110813218
----------cost is ------------- 0.31008458990714444
----------cost is ------------- 0.26884149761424625
----------cost is ------------- 0.30775652324711167
----------cost is ------------- 0.2669142677622906
----------cost is ------------- 0.3054370838167704
----------cost is ------------- 0.26499382269412697
----------cost is ------------- 0.3031259792387094
----------cost is ------------- 0.263079974421909
----------cost is ------------- 0.30082293385910286
----------cost is ------------- 0.2611725468149768
----------cost is ------------- 0.29852769005246993
----------cost is ------------- 0.2592713765119504
----------cost is ------------- 0.29624000933883177
----------cost is ------------- 0.25737631367036057
----------cost is ------------- 0.2939596732323898
----------cost is ----

...

----------cost is ------------- 0.020137995023357932
----------cost is ------------- 0.020115168779292092
----------cost is ------------- 0.02009238683290437
----------cost is ------------- 0.020069649066419646
----------cost is ------------- 0.02004695536244527
----------cost is ------------- 0.020024305603969585
----------cost is ------------- 0.02000169967436052
----------cost is ------------- 0.019979137457364134
----------cost is ------------- 0.01995661883710317
----------cost is ------------- 0.019934143698075647
----------cost is ------------- 0.019911711925153456
----------cost is ------------- 0.019889323403580905
----------cost is ------------- 0.019866978018973365
----------cost is ------------- 0.019844675657315847
----------cost is ------------- 0.019822416204961616
----------cost is ------------- 0.019800199548630833
----------cost is ------------- 0.019778025575409135
----------cost is ------------- 0.019755894172746334
----------cost is ------------- 0.0197338052284549

----------cost is ------------- 0.01704771306838616
----------cost is ------------- 0.017030580728415765
----------cost is ------------- 0.017013478178009425
----------cost is ------------- 0.01699640534502335
----------cost is ------------- 0.01697936215753276
----------cost is ------------- 0.01696234854383106
----------cost is ------------- 0.016945364432429128
----------cost is ------------- 0.01692840975205452
----------cost is ------------- 0.01691148443165074
----------cost is ------------- 0.016894588400376487
----------cost is ------------- 0.016877721587604925
----------cost is ------------- 0.016860883922922867
----------cost is ------------- 0.016844075336130165
----------cost is ------------- 0.01682729575723881
----------cost is ------------- 0.016810545116472318
----------cost is ------------- 0.01679382334426496
----------cost is ------------- 0.016777130371261
----------cost is ------------- 0.016760466128314014
----------cost is ------------- 0.016743830546486108
----

----------cost is ------------- 0.014695144847144782
----------cost is ------------- 0.014681907097561387
----------cost is ------------- 0.014668690116557515
----------cost is ------------- 0.014655493858437904
----------cost is ------------- 0.014642318277635404
----------cost is ------------- 0.01462916332871057
----------cost is ------------- 0.014616028966351207
----------cost is ------------- 0.014602915145371995
----------cost is ------------- 0.014589821820713975
----------cost is ------------- 0.01457674894744422
----------cost is ------------- 0.014563696480755362
----------cost is ------------- 0.014550664375965172
----------cost is ------------- 0.014537652588516164
----------cost is ------------- 0.014524661073975157
----------cost is ------------- 0.014511689788032866
----------cost is ------------- 0.014498738686503504
----------cost is ------------- 0.014485807725324337
----------cost is ------------- 0.014472896860555301
----------cost is ------------- 0.01446000604837

----------cost is ------------- 0.012866041958088398
----------cost is ------------- 0.012855547038737232
----------cost is ------------- 0.012845067106133486
----------cost is ------------- 0.01283460213016546
----------cost is ------------- 0.012824152080798944
----------cost is ------------- 0.012813716928076938
----------cost is ------------- 0.012803296642119467
----------cost is ------------- 0.012792891193123254
----------cost is ------------- 0.012782500551361587
----------cost is ------------- 0.012772124687184
----------cost is ------------- 0.01276176357101607
----------cost is ------------- 0.012751417173359177
----------cost is ------------- 0.012741085464790254
----------cost is ------------- 0.012730768415961583
----------cost is ------------- 0.012720465997600532
----------cost is ------------- 0.012710178180509322
----------cost is ------------- 0.012699904935564837
----------cost is ------------- 0.01268964623371833
----------cost is ------------- 0.012679402045995231

----------cost is ------------- 0.011393015810768915
----------cost is ------------- 0.01138453769371083
----------cost is ------------- 0.011376070671192775
----------cost is ------------- 0.011367614722737718
----------cost is ------------- 0.011359169827917109
----------cost is ------------- 0.011350735966350759
----------cost is ------------- 0.011342313117706712
----------cost is ------------- 0.011333901261701067
----------cost is ------------- 0.011325500378097902
----------cost is ------------- 0.011317110446709069
----------cost is ------------- 0.01130873144739409
----------cost is ------------- 0.011300363360060035
----------cost is ------------- 0.011292006164661347
----------cost is ------------- 0.011283659841199743
----------cost is ------------- 0.011275324369724053
----------cost is ------------- 0.011266999730330097
----------cost is ------------- 0.011258685903160552
----------cost is ------------- 0.01125038286840481
----------cost is ------------- 0.011242090606298

In [27]:
################# 先用 验证集进行验证 ##################
test_error_count = 0
#print(test_data.shape[0])
for i in range(test_data.shape[0]-1):
    x_test_data = test_data[i]
    hid_input = np.matmul(x_test_data,w1)
    hid_output = np.tanh(hid_input)
    hid_output = np.insert(hid_output,0,1,axis=0)
    hpfs_input = np.matmul(hid_output,w2)
    hpfs_output = sigmond(hpfs_input)
    #print(y_test.shape)
    if (hpfs_output>0.5 and y_test[i] ==0 ) or (hpfs_output <0.5 and y_test[i] ==1):
        test_error_count = test_error_count +1
        print(hpfs_output,y_test[i])
print(test_error_count/test_data.shape[0])

[0.16730154] [1]
[0.8730742] [0]
[0.42537996] [1]
[0.84155627] [0]
[1.31175975e-05] [1]
[0.2294125] [1]
[0.00377776] [1]
[0.09072088] [1]
[0.55263283] [0]
[0.71949034] [0]
[0.95142828] [0]
[0.9993505] [0]
[0.47181639] [1]
[0.99962795] [0]
[0.98325527] [0]
[0.97250162] [0]
[0.05538653] [1]
[0.55431468] [0]
[0.99931254] [0]
[0.00307653] [1]
[0.99864202] [0]
[0.0003459] [1]
[0.70839154] [0]
[0.88772391] [0]
[0.6880096] [0]
[0.01947343] [1]
[0.00066985] [1]
[0.81679641] [0]
[0.99158716] [0]
[0.99767994] [0]
[0.69788955] [0]
[0.58895953] [0]
[0.008851] [1]
[0.82244351] [0]
[0.02027431] [1]
[0.09090247] [1]
[0.99472798] [0]
[0.03357979] [1]
[0.53157486] [0]
[0.25844207] [1]
[0.91359771] [0]
[0.99848317] [0]
[0.98806219] [0]
[0.19384218] [1]
[0.99989144] [0]
[0.6674839] [0]
[0.99504196] [0]
[0.25116821] [1]
[0.00293839] [1]
[0.85633661] [0]
[0.28136277] [1]
[0.00291471] [1]
[0.75962552] [0]
[0.44561398] [1]
[0.54854141] [0]
[0.99945497] [0]
[0.91008167] [0]
[0.35132978] [1]
[0.03389009] [1]
[

In [35]:
################# 先用 验证集进行验证 ##################
all_error_count = 0
#all_x_data = np.insert(all_x_data,0,1,axis=1)
#print(test_data.shape[0])
for i in range(all_x_data.shape[0]-1):
    x_test_data = all_x_data[i]
    hid_input = np.matmul(x_test_data,w1)
    hid_output = np.tanh(hid_input)
    hid_output = np.insert(hid_output,0,1,axis=0)
    hpfs_input = np.matmul(hid_output,w2)
    hpfs_output = sigmond(hpfs_input)
    #print(y_test.shape)
    if (hpfs_output>0.5 and stock_lable_org.iloc[i,2] ==0 ) or (hpfs_output <0.5 and stock_lable_org.iloc[i,2] ==1):
        all_error_count = all_error_count +1
        print(hpfs_output,stock_lable_org.iloc[i,2])
print(all_error_count/all_x_data.shape[0])

[0.99978577] 0
[0.99732817] 0
[0.96121346] 0
[0.99975277] 0
[0.98372549] 0
[0.52453424] 0
[0.9915243] 0
[0.64914297] 0
[0.98469243] 0
[0.70973355] 0
[0.98756794] 0
[0.64491854] 0
[0.99885914] 0
[0.99723177] 0
[0.56817425] 0
[0.76700883] 0
[0.99475693] 0
[0.88177927] 0
[0.63313657] 0
[0.99936041] 0
[0.99963167] 0
[0.95408646] 0
[0.99922409] 0
[0.99985709] 0
[0.56901567] 0
[0.91771782] 0
[0.92069467] 0
[0.99996245] 0
[0.87475803] 0
[0.99934279] 0
[0.99234794] 0
[0.94379816] 0
[0.76649328] 0
[0.98941226] 0
[0.58942865] 0
[0.99337356] 0
[0.99784362] 0
[0.96612888] 0
[0.99844181] 0
[0.60461723] 0
[0.99966229] 0
[0.99843225] 0
[0.74046223] 0
[0.89269281] 0
[0.90428596] 0
[0.92908138] 0
[0.89076847] 0
[0.99864748] 0
[0.99289879] 0
[0.98486393] 0
[0.99810364] 0
[0.94284403] 0
[0.99999284] 0
[0.99500211] 0
[0.90948344] 0
[0.99985462] 0
[0.77453505] 0
[0.58974917] 0
[0.99837845] 0
[0.82472784] 0
[0.98796202] 0
[0.9735263] 0
[0.79349349] 0
[0.99886817] 0
[0.98347153] 0
[0.75892451] 0
[0.99412229]

[0.97568864] 0
[0.99997771] 0
[0.85873403] 0
[0.95036328] 0
[0.99269959] 0
[0.99973105] 0
[0.99989372] 0
[0.78279716] 0
[0.9969985] 0
[0.99947809] 0
[0.76986445] 0
[0.99983285] 0
[0.71399055] 0
[0.98657041] 0
[0.84877245] 0
[0.78466635] 0
[0.97132184] 0
[0.73249464] 0
[0.94906126] 0
[0.9831952] 0
[0.80043918] 0
[0.80087159] 0
[0.98327553] 0
[0.98288906] 0
[0.96970877] 0
[0.98896455] 0
[0.93508132] 0
[0.5616035] 0
[0.9860245] 0
[0.99679429] 0
[0.9987679] 0
[0.99725403] 0
[0.9766289] 0
[0.98694039] 0
[0.97812666] 0
[0.78906664] 0
[0.54492101] 0
[0.88912125] 0
[0.99988414] 0
[0.90438932] 0
[0.99759393] 0
[0.88886755] 0
[0.61306675] 0
[0.63295132] 0
[0.99659326] 0
[0.65564997] 0
[0.94304394] 0
[0.77760353] 0
[0.88172908] 0
[0.92472609] 0
[0.99996348] 0
[0.9965952] 0
[0.84615483] 0
[0.99290319] 0
[0.94389094] 0
[0.97551951] 0
[0.60151008] 0
[0.8370446] 0
[0.9973586] 0
[0.99997371] 0
[0.66693638] 0
[0.89450374] 0
[0.99986697] 0
[0.82756855] 0
[0.98288259] 0
[0.66546622] 0
[0.99880722] 0
[0.9

[0.52301187] 0
[0.8220259] 0
[0.81886632] 0
[0.99741229] 0
[0.98823871] 0
[0.99979199] 0
[0.99911645] 0
[0.85658619] 0
[0.89335219] 0
[0.83239341] 0
[0.99793786] 0
[0.74196557] 0
[0.72358424] 0
[0.64156384] 0
[0.91325398] 0
[0.99418458] 0
[0.93828192] 0
[0.91028355] 0
[0.98739031] 0
[0.99910849] 0
[0.61897546] 0
[0.998902] 0
[0.83455971] 0
[0.99856804] 0
[0.97945701] 0
[0.97688934] 0
[0.97567617] 0
[0.99824454] 0
[0.99303506] 0
[0.9231792] 0
[0.92192014] 0
[0.98719202] 0
[0.63067055] 0
[0.98635883] 0
[0.98252714] 0
[0.9864999] 0
[0.9477299] 0
[0.73341355] 0
[0.53454881] 0
[0.99716744] 0
[0.69110111] 0
[0.94046462] 0
[0.99966984] 0
[0.99889011] 0
[0.90141573] 0
[0.99997632] 0
[0.97558268] 0
[0.82396261] 0
[0.8162305] 0
[0.99008183] 0
[0.58752925] 0
[0.74112295] 0
[0.54527445] 0
[0.82601103] 0
[0.96805424] 0
[0.99552531] 0
[0.99998967] 0
[0.58857773] 0
[0.52135252] 0
[0.99995323] 0
[0.9202126] 0
[0.97852952] 0
[0.99731393] 0
[0.96183667] 0
[0.52715738] 0
[0.94395607] 0
[0.99769863] 0
[0.

[0.64608391] 0
[0.82005963] 0
[0.98453369] 0
[0.99961044] 0
[0.99869196] 0
[0.99830628] 0
[0.91351957] 0
[0.65644741] 0
[0.99993583] 0
[0.60387812] 0
[0.56977728] 0
[0.99999247] 0
[0.94614288] 0
[0.67247133] 0
[0.6781651] 0
[0.98648023] 0
[0.66095812] 0
[0.99547711] 0
[0.7945506] 0
[0.97419432] 0
[0.94774109] 0
[0.99160326] 0
[0.72617913] 0
[0.76218962] 0
[0.7684529] 0
[0.99079084] 0
[0.85240863] 0
[0.96778584] 0
[0.9678414] 0
[0.76205378] 0
[0.99943125] 0
[0.99345659] 0
[0.87895713] 0
[0.9467592] 0
[0.8041888] 0
[0.67956388] 0
[0.61638238] 0
[0.96702869] 0
[0.999989] 0
[0.91693367] 0
[0.99014647] 0
[0.98954438] 0
[0.87205873] 0
[0.55955769] 0
[0.95917967] 0
[0.68239257] 0
[0.99918987] 0
[0.9979659] 0
[0.86322576] 0
[0.9999915] 0
[0.99874647] 0
[0.99825047] 0
[0.67361361] 0
[0.99988285] 0
[0.99141588] 0
[0.99521232] 0
[0.78391053] 0
[0.90390732] 0
[0.82255699] 0
[0.99665328] 0
[0.98218685] 0
[0.9248389] 0
[0.9578292] 0
[0.99382059] 0
[0.99858915] 0
[0.9984019] 0
[0.99983465] 0
[0.97450

[0.58492464] 0
[0.94016463] 0
[0.69476458] 0
[0.99581252] 0
[0.97444013] 0
[0.87701114] 0
[0.78985828] 0
[0.98838627] 0
[0.695707] 0
[0.98497275] 0
[0.83680589] 0
[0.84783715] 0
[0.99940167] 0
[0.87844946] 0
[0.64566833] 0
[0.99712077] 0
[0.9991697] 0
[0.59939518] 0
[0.8749581] 0
[0.89049153] 0
[0.71028353] 0
[0.84066676] 0
[0.98922136] 0
[0.8446353] 0
[0.99999711] 0
[0.99998618] 0
[0.89769147] 0
[0.99999892] 0
[0.54628556] 0
[0.88701958] 0
[0.99981469] 0
[0.99805187] 0
[0.9935569] 0
[0.81612708] 0
[0.99559971] 0
[0.99996119] 0
[0.9406137] 0
[0.71468163] 0
[0.981473] 0
[0.99867462] 0
[0.97417246] 0
[0.7377349] 0
[0.99751653] 0
[0.94721369] 0
[0.95407048] 0
[0.55208492] 0
[0.99675182] 0
[0.89084646] 0
[0.99999928] 0
[0.9997679] 0
[0.99911234] 0
[0.99975842] 0
[0.56568117] 0
[0.76185115] 0
[0.98252074] 0
[0.99955436] 0
[0.82122166] 0
[0.72889517] 0
[0.97691017] 0
[0.94074546] 0
[0.99947368] 0
[0.89634384] 0
[0.99999661] 0
[0.96251949] 0
[0.59285899] 0
[0.99994496] 0
[0.99971395] 0
[0.730

[0.98012258] 0
[0.72663701] 0
[0.8445596] 0
[0.99998215] 0
[0.99804151] 0
[0.99968266] 0
[0.99364349] 0
[0.9954673] 0
[0.0002739] 1
[0.56018982] 0
[0.91826084] 0
[0.55127021] 0
[0.96983147] 0
[0.99992885] 0
[0.65464893] 0
[0.93616972] 0
[0.68238973] 0
[0.97049506] 0
[0.96130918] 0
[0.91543887] 0
[0.84115851] 0
[0.91066157] 0
[0.99969901] 0
[0.72523007] 0
[0.98200986] 0
[0.99983518] 0
[0.9999993] 0
[0.12527353] 1
[0.99975356] 0
[0.99840569] 0
[0.90423795] 0
[0.999807] 0
[0.85966568] 0
[0.98309938] 0
[0.65567495] 0
[0.99715761] 0
[0.71327678] 0
[0.95792467] 0
[0.00011897] 1
[0.54967724] 0
[0.65440817] 0
[0.97083705] 0
[0.99912535] 0
[0.98773045] 0
[0.96732853] 0
[0.97053962] 0
[0.86289649] 0
[0.63514952] 0
[0.46817375] 1
[0.74300248] 0
[0.99653329] 0
[0.88288408] 0
[0.99999147] 0
[0.80048724] 0
[0.32355955] 1
[0.99306753] 0
[0.61437528] 0
[0.41986491] 1
[0.8143257] 0
[0.86833412] 0
[0.87594929] 0
[0.99903406] 0
[0.85030887] 0
[0.99999934] 0
[0.97231861] 0
[0.99928865] 0
[0.96507317] 0
[0

[0.83640005] 0
[0.98699402] 0
[0.9025967] 0
[0.60185602] 0
[0.99913674] 0
[0.62218765] 0
[0.83859457] 0
[0.79394069] 0
[0.98891094] 0
[0.99457199] 0
[0.99836484] 0
[0.99998864] 0
[0.99968219] 0
[0.98835565] 0
[0.75846064] 0
[0.71530475] 0
[0.99868393] 0
[0.61867608] 0
[0.55893358] 0
[0.90829608] 0
[0.89145283] 0
[0.94668774] 0
[0.96838742] 0
[0.79382312] 0
[0.88875717] 0
[0.76310936] 0
[0.9511444] 0
[0.85587937] 0
[0.99802635] 0
[0.9983499] 0
[0.99088635] 0
[0.85573111] 0
[0.99851551] 0
[0.52612218] 0
[0.9999986] 0
[0.99947732] 0
[0.98785042] 0
[0.68854083] 0
[0.9666132] 0
[0.95230287] 0
[0.96894219] 0
[0.52124275] 0
[0.99976728] 0
[0.99598526] 0
[0.50350989] 0
[0.99850447] 0
[0.99984201] 0
[0.99992093] 0
[0.9998186] 0
[0.67128352] 0
[0.62244867] 0
[0.60314693] 0
[0.74315424] 0
[0.71492275] 0
[0.78977039] 0
[0.78717919] 0
[0.99996249] 0
[0.9996942] 0
[0.97859731] 0
[0.93202669] 0
[0.68776893] 0
[0.92281806] 0
[0.94202695] 0
[0.99986957] 0
[0.55729717] 0
[0.73911281] 0
[0.9972744] 0
[0.